# Sistema de aquisição de dados de voz normal e sussurrada

Criado por: Cezar Yamamura, Rogério Pignelli, Prof. Dr. Paulo Scalassara

Data: 19 set 2024

In [ ]:
import sounddevice as sd
import soundfile as sf
import ipywidgets as widgets
from IPython.display import display, clear_output
import os
import numpy as np

# Configurações de gravação
sample_rate = 44100  # 44.1kHz
channels = 1
recording = None  # Variável global para armazenar o áudio gravado
current_index = 0  # Índice da palavra atual
mode = 'normal'  # Modo de gravação: 'normal' ou 'sussurrada'
audio_index = 0 # descrição dos audios
is_recording = False
recording = []

# Lista de palavras
words = ["Boia", "Frente"]

# Função de callback para capturar o áudio continuamente
def callback(indata, frames, time, status):
    if is_recording:
        recording.append(indata.copy())

# Função para iniciar a gravação
def start_recording(b):
    global recording, is_recording, stream
    recording = []  # Reseta a gravação anterior
    is_recording = True
    print(f"Iniciando a gravação da palavra '{words[current_index]}' com fala {mode}...")
    # Inicia o fluxo de entrada com callback
    stream = sd.InputStream(samplerate=sample_rate, channels=channels, callback=callback)
    stream.start()  

# Função para parar a gravação e salvar o arquivo
def stop_recording(b):
    global recording, current_index, mode, stream, is_recording
    is_recording = False
    stream.stop()
    stream.close()
    print("Gravação parada.")
    # Concatena todos os segmentos gravados
    audio_data = np.concatenate(recording, axis=0)

    word = words[current_index]
    folder = f"fala_{mode}"
    filename = os.path.join(folder, f"{word}_{mode}.flac")
    if not os.path.exists(folder):
        os.makedirs(folder)
    sf.write(filename, audio_data, sample_rate, format='FLAC')
    print(f"Gravação da palavra '{word}' com fala {mode} salva como {filename}")
    
# Função para avançar para a próxima palavra
def next_word(b):
    global current_index, mode, audio_index
    clear_output()  # Limpa a saída do notebook

    # Alterna entre fala normal e sussurrada
    if mode == 'normal':
        mode = 'sussurrada'
    else:
        mode = 'normal'
        current_index += 1  # Passa para a próxima palavra
        audio_index += 1 

    if current_index < len(words):
        print(f"Grave a palavra '{words[current_index]}' com fala {mode}:")
        display(start_button, stop_button, next_button)
    else:
        print("Todas as palavras foram gravadas.")
        current_index = 0  # Reinicia a contagem se necessário

 # Função para iniciar o sistema
def start_system(b):
    global current_index, mode, audio_index
    clear_output()  # Limpa a saída do notebook

    print(f"Grave a palavra '{words[current_index]}' com fala {mode}:")
    display(start_button, stop_button, next_button)


# Botões para iniciar, parar a gravação e avançar para a próxima palavra
start_button = widgets.Button(description="Iniciar Gravação")
stop_button = widgets.Button(description="Parar Gravação")
next_button = widgets.Button(description="Próximo")
start_system_button = widgets.Button(description="Start System")


# Atribui funções aos botões
start_button.on_click(start_recording)
stop_button.on_click(stop_recording)
next_button.on_click(next_word)
start_system_button.on_click(start_system)

# Inicia a interface
print("Clique em 'Próximo' para iniciar a gravação:")
display(start_system_button)



Todas as palavras foram gravadas.
